# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 

from recfldtkn.ckpd_obs import Ckpd_ObservationS
from recfldtkn.configfn import load_cohort_args
from recfldtkn.obsname import convert_RecObsName_and_CaseTkn_to_CaseObsName
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/jluo/Desktop/2024-DrFirst-SPACE/_DrFirst-AI-EduRxPred-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_EduRxPred',
 'MODEL_TASK': 'Model_EduRxPred',
 'WORKSPACE_PATH': '/Users/jluo/Desktop/2024-DrFirst-SPACE/_DrFirst-AI-EduRxPred-WorkSpace'}


/Users/jluo/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# [Part 1] Get a Caseset: Case Examples

## [Step 1]

In [2]:
from recfldtkn.aidstools import get_caseset_to_observe

####################
caseset_name = 'TrulicityRx'
group_id = 0
case_id_columns = ["PID", "ObsDT", 'PInvID', 'RxID']
####################

CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder', caseset_name)
print(CaseFolder)
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['Ckpd_ObservationS'] = Ckpd_ObservationS
print(cohort_args)

group_name, ds_case = get_caseset_to_observe(group_id, CaseFolder, case_id_columns, cohort_args)
print(group_name)
print(ds_case)

Data_EduRxPred/CaseFolder/TrulicityRx
{'RawRootID': 'patient_id_encoded', 'CohortInfo': {'RawData2023Nov_Trulicity': {'cohort_label': 1, 'cohort_name': 'RawData2023Nov_Trulicity', 'FolderPath': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity'}}, 'RootID': 'PID', 'RecName': 'PRawRecNum', 'rec_folder': '../_Data/1-Data_RFT/RecAttr/', 'fld_folder': '../_Data/1-Data_RFT/RecAttr/', 'hfds_folder': '../_Data/1-Data_RFT/HfDataset/', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'RecName_to_RFT_GROUP_SIZE': {'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'Default': 10000}, 'RecName_to_RFT_usebucket': {'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_path': '../pipeline/config_recfldtkn/', 'SPACE': {'DATA_RAW': '../_Data/0-Data_Raw', 'DATA_RFT': '../_Data/1-Data_RFT', 'DATA_CaseObs': '../_Data/2-Data_CaseObs', 'CODE_FN': '../pipeline', 'CODE_RFT': '../pipeline', 'DATA_TASK': 'Data_EduRxPre


## [Step 2] RO: Record Observation & CaseTkn


In [8]:
from recfldtkn.obsname import convert_RecObsName_and_CaseTkn_to_CaseObsName

###########################
Record_Observations_List = ['EgmEdu-Af1W']
CaseTkn = 'FutRxEduTkn'
###########################

CaseObsName = convert_RecObsName_and_CaseTkn_to_CaseObsName(Record_Observations_List, CaseTkn)
# CO_ij
print(CaseObsName)

ro.EgmEdu-Af1W_ct.FutRxEduTkn


In [9]:
CaseObsFolder = os.path.join(SPACE['DATA_CaseObs'], CaseObsName)
if not os.path.exists(CaseObsFolder):
    os.makedirs(CaseObsFolder)
    
CaseObsFolder_data = os.path.join(CaseObsFolder, 'data')    
CaseObsFolder_vocab = os.path.join(CaseObsFolder, 'vocab.p')

print(CaseObsFolder_data) 
print(CaseObsFolder_vocab)

../_Data/2-Data_CaseObs/ro.EgmEdu-Af1W_ct.FutRxEduTkn/data
../_Data/2-Data_CaseObs/ro.EgmEdu-Af1W_ct.FutRxEduTkn/vocab.p


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [10]:
case_examples = ds_case.shuffle(seed=42).select(range(10))[:10]
print(case_examples)

{'PID': [1024150, 1013126, 1034286, 1012301, 1038570, 1032487, 1042644, 1042660, 1032203, 1009855], 'ObsDT': [Timestamp('2023-11-28 15:55:59.016000'), Timestamp('2023-06-15 02:29:33.640000'), Timestamp('2023-09-29 13:40:46.160000'), Timestamp('2023-06-07 13:23:15.061000'), Timestamp('2023-06-13 15:59:16.918000'), Timestamp('2023-09-01 08:39:11.971000'), Timestamp('2023-04-04 18:46:16.428000'), Timestamp('2023-09-27 17:42:56.103000'), Timestamp('2023-09-21 00:21:53.416000'), Timestamp('2023-05-17 18:14:31.324000')], 'PInvID': ['1024150-007', '1013126-009', '1034286-002', '1012301-008', '1038570-008', '1032487-000', '1042644-001', '1042660-012', '1032203-000', '1009855-003'], 'RxID': ['1024150-007-001', '1013126-009-000', '1034286-002-000', '1012301-008-001', '1038570-008-004', '1032487-000-000', '1042644-001-000', '1042660-012-000', '1032203-000-001', '1009855-003-000']}


In [11]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1024150,
     'ObsDT': Timestamp('2023-11-28 15:55:59.016000'),
     'PInvID': '1024150-007',
     'RxID': '1024150-007-001'},
 1: {'PID': 1013126,
     'ObsDT': Timestamp('2023-06-15 02:29:33.640000'),
     'PInvID': '1013126-009',
     'RxID': '1013126-009-000'},
 2: {'PID': 1034286,
     'ObsDT': Timestamp('2023-09-29 13:40:46.160000'),
     'PInvID': '1034286-002',
     'RxID': '1034286-002-000'},
 3: {'PID': 1012301,
     'ObsDT': Timestamp('2023-06-07 13:23:15.061000'),
     'PInvID': '1012301-008',
     'RxID': '1012301-008-001'},
 4: {'PID': 1038570,
     'ObsDT': Timestamp('2023-06-13 15:59:16.918000'),
     'PInvID': '1038570-008',
     'RxID': '1038570-008-004'},
 5: {'PID': 1032487,
     'ObsDT': Timestamp('2023-09-01 08:39:11.971000'),
     'PInvID': '1032487-000',
     'RxID': '1032487-000-000'},
 6: {'PID': 1042644,
     'ObsDT': Timestamp('2023-04-04 18:46:16.428000'),
     'PInvID': '1042644-001',
     'RxID': '1042644-001-000'},
 7: {'PID': 1042660,
     '

In [12]:
case_example = idx_to_examples[0]
case_example

{'PID': 1024150,
 'ObsDT': Timestamp('2023-11-28 15:55:59.016000'),
 'PInvID': '1024150-007',
 'RxID': '1024150-007-001'}

## [Step 2] get_caseobs_id

In [13]:
# InObs - Rx - RecNum = 10

# CO_id
# CO: RO_RecNUm
# CO: PZipInfo COid # PID
# CO^{recnum}, CO^{futeduegm}

# {'PID': 1020788,
#  'ObsDT': Timestamp('2023-09-06 17:41:01.757000'),
#  'PInvID': '1020788-000',
#  'RxID': '1020788-000-000'} # <--- edu future engagement

# {'PID': 1020788,
#  'ObsDT': Timestamp('2023-09-06 17:41:01.757000'),
#  'PInvID': '1020788-000',
#  'RxID': '1020788-000-001'} # <--- edu future engagement

In [16]:
import inspect
#################################################
def get_caseobs_id(case_example, CaseObsName):
    PIDValue = case_example['PID']
    ObsDTValue = case_example['ObsDT'].isoformat()
    RxIDValue = case_example['RxID']
    return f'{PIDValue}-{ObsDTValue}-{RxIDValue}'
get_caseobs_id.fn_string = inspect.getsource(get_caseobs_id)
#################################################

In [17]:
caseobs_id = get_caseobs_id(case_example, CaseObsName)
caseobs_id # CO_id

'1024150-2023-11-28T15:55:59.016000-1024150-007-001'



## [Step 3] RO List and CaseTkn

In [18]:
CaseTkn

'FutRxEduTkn'

In [19]:
# cohort_args

In [20]:
## we need to prepare these five arguments.
# CaseTkn     # <------- we have this.
# cohort_args # <------- we have this.

# RecObs_Name 
# rec_args
# column_names # attribute name of R^recordname

In [21]:
######### RecObs_Name
print(Record_Observations_List) # RO_List
# Let's focus on one RO
RO = Record_Observations_List[0]
RecObs_Name = RO
print(RecObs_Name)

['EgmEdu-Af1W']
EgmEdu-Af1W


In [22]:
# RO = 'Rx-Bf2M-InsCate'
# RO = 'Rx-Bf2M'
# RO = 'P-Zip3DemoNume'

## [Step4] RO Information

In [23]:
# decompose RO to RecName, CkpdName, FldName (small phi)
from recfldtkn.obsname import parse_RecObsName

d = parse_RecObsName(RecObs_Name)
RecName = d['RecName']
CkpdName = d['CkpdName']
FldName = d['FldName']
print(RecName, CkpdName, FldName)

EgmEdu Af1W None


In [24]:
# load all information about the R^record_name. 
from recfldtkn.configfn import load_record_args

print(RecName)
rec_args = load_record_args(RecName, cohort_args)
# TODO: interpert each keys in the rec_args. Put them in the miro. 
[i for i in rec_args]

EgmEdu


['CohortInfo',
 'RecName',
 'RecID',
 'RecIDChain',
 'ParentRecName',
 'RecDT',
 'attr_cols',
 'pypath',
 'FldTknInfo',
 'RFT_GROUP_SIZE',
 'idx_group_size',
 'usebucket',
 'GROUP_SIZE',
 'folder',
 'rec_folder',
 'recfldtkn_config_path',
 'yaml_file_path']

In [25]:
# rec_args['attr_cols']

## [Step 5] Load ds_rec

In [26]:
from recfldtkn.loadtools import load_ds_rec_and_info

ds_rec, ds_rec_info = load_ds_rec_and_info(RecName, cohort_args)

# R^Rx
# ROdf # Rx-Bf2M (i, t, Bf2M)

In [27]:
######### column_names (or we can all it attribute columns)
column_names = ds_rec.column_names # attr_columns + phi_columns
column_names
# ROdf_ij --(Phi: RecNum)-> COdf_ij 

['PID',
 'PInvID',
 'RxID',
 'EgmEduID',
 'event_id',
 'DT',
 'invitation_date',
 'event_date',
 'server_id',
 'user_agent']

## [Step 6] get_selected_columns (attr + phi) from ds_rec

In [28]:
###############################
def get_selected_columns(RecObs_Name, column_names, cohort_args, rec_args, CaseTkn):
    # column_names = [i for i in column_names if 'Tkn_' not in i]
    base_columns = rec_args['RecIDChain'] + [rec_args['RecDT']]
    return base_columns

get_selected_columns.fn_string = inspect.getsource(get_selected_columns)
###############################

In [29]:
# big question here
# if small phi is not applied in the record level
# how do we deploy the tools of small phi within this process?

get_selected_columns(RecObs_Name, column_names, cohort_args, rec_args, CaseTkn)

['PID', 'PInvID', 'RxID', 'EgmEduID', 'DT']

In [30]:
print(Record_Observations_List)
print(CaseTkn)
print(get_selected_columns)

['EgmEdu-Af1W']
FutRxEduTkn
<function get_selected_columns at 0x13fca9120>


In [31]:
# RecObsName_to_RecObsInfo # RO_to_ROinfo

from recfldtkn.observer import get_RecObsName_to_RecObsInfo

# this function is the wrapper of the above cells.
# RO_to_ROInfo 
RecObsName_to_RecObsInfo = get_RecObsName_to_RecObsInfo(Record_Observations_List, 
                                                        CaseTkn, get_selected_columns, 
                                                        cohort_args, Ckpd_ObservationS)


RO = Record_Observations_List[0] # Rx-Bf2M  
ROInfo = RecObsName_to_RecObsInfo[RO] # RxInfo + R^Rx (with necessary columns) + Ckpd + FldTkn (phi)


In [32]:
[i for i in ROInfo]

# pprint(ROInfo)

['rec_args',
 'RecName',
 'CkpdName',
 'CkpdInfo',
 'FldName',
 'FldTknName',
 'FldIdx2Tkn',
 'ds_rec',
 'df_rec_info']

In [33]:
ROInfo['ds_rec'] # get_selected_columns

Dataset({
    features: ['PID', 'PInvID', 'RxID', 'EgmEduID', 'DT'],
    num_rows: 4521
})

## [Step 7] get_casetkn_vocab

In [34]:
import inspect

######################################
def get_casetkn_vocab(RecObsName_to_RecObsInfo):
    ############################
    idx2tkn = ['unk', 
               'InvBtnEgm', 'InvBtnEgmClicks', 'InvMinsUntil1stEgm', 
               'RxBtnEgm', 'RxBtnEgmClicks', 'RxMinsUntil1stEgm' ]
    ############################

    tid2tkn = {tid: tkn for tid, tkn in enumerate(idx2tkn)}
    tkn2tid = {tkn: tid for tid, tkn in tid2tkn.items()}
    CaseTknVocab = {}
    CaseTknVocab['tkn'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    return CaseTknVocab
######################################

get_casetkn_vocab.fn_string = inspect.getsource(get_casetkn_vocab)

In [35]:
CaseTknVocab = get_casetkn_vocab(RecObsName_to_RecObsInfo)
pprint(CaseTknVocab, sort_dicts=False)

{'tkn': {'tid2tkn': {0: 'unk',
                     1: 'InvBtnEgm',
                     2: 'InvBtnEgmClicks',
                     3: 'InvMinsUntil1stEgm',
                     4: 'RxBtnEgm',
                     5: 'RxBtnEgmClicks',
                     6: 'RxMinsUntil1stEgm'},
         'tkn2tid': {'unk': 0,
                     'InvBtnEgm': 1,
                     'InvBtnEgmClicks': 2,
                     'InvMinsUntil1stEgm': 3,
                     'RxBtnEgm': 4,
                     'RxBtnEgmClicks': 5,
                     'RxMinsUntil1stEgm': 6}}}



## [Step 8] Fetch ds_rec: $R^{recname}$

In [40]:
case_example = idx_to_examples[4]
case_example

{'PID': 1038570,
 'ObsDT': Timestamp('2023-06-13 15:59:16.918000'),
 'PInvID': '1038570-008',
 'RxID': '1038570-008-004'}

In [41]:
RO

'EgmEdu-Af1W'

In [42]:
# RO_to_ROdf: RecObsName_to_RecObsDS
from recfldtkn.observer import get_Record_P

pprint(idx_to_examples, sort_dicts=False)

RecName_to_REC_P = get_Record_P(idx_to_examples, # a list of cases
                                RecObsName_to_RecObsInfo, # RO_to_ROinfo
                                )

pprint(RecName_to_REC_P) # R_i^{RecName}

{0: {'PID': 1024150,
     'ObsDT': Timestamp('2023-11-28 15:55:59.016000'),
     'PInvID': '1024150-007',
     'RxID': '1024150-007-001'},
 1: {'PID': 1013126,
     'ObsDT': Timestamp('2023-06-15 02:29:33.640000'),
     'PInvID': '1013126-009',
     'RxID': '1013126-009-000'},
 2: {'PID': 1034286,
     'ObsDT': Timestamp('2023-09-29 13:40:46.160000'),
     'PInvID': '1034286-002',
     'RxID': '1034286-002-000'},
 3: {'PID': 1012301,
     'ObsDT': Timestamp('2023-06-07 13:23:15.061000'),
     'PInvID': '1012301-008',
     'RxID': '1012301-008-001'},
 4: {'PID': 1038570,
     'ObsDT': Timestamp('2023-06-13 15:59:16.918000'),
     'PInvID': '1038570-008',
     'RxID': '1038570-008-004'},
 5: {'PID': 1032487,
     'ObsDT': Timestamp('2023-09-01 08:39:11.971000'),
     'PInvID': '1032487-000',
     'RxID': '1032487-000-000'},
 6: {'PID': 1042644,
     'ObsDT': Timestamp('2023-04-04 18:46:16.428000'),
     'PInvID': '1042644-001',
     'RxID': '1042644-001-000'},
 7: {'PID': 1042660,
     '

In [43]:
# RecName_to_REC_P['Rx'][1020788]

## [Step 9] Fetch ROdf: $R_i^{recname}$

In [44]:
from recfldtkn.observer import get_idx_to_RecObsName_to_RecObsDS
idx_to_RO_to_ROdf = get_idx_to_RecObsName_to_RecObsDS(idx_to_examples, # a list of cases
                                                       RecObsName_to_RecObsInfo, # RO_to_ROinfo
                                                       RecName_to_REC_P, # R_i^{recname}
                                                       )

for idx, case in idx_to_examples.items():
    print('\n====================')
    print('idx:', idx)
    print('case:', case)
    print('ROdf:', idx_to_RO_to_ROdf[idx]) # R_i^{RecNum, Ckpd}
    print('====================')

# RO_list = [RO]
    
# RO --> {RecName, Ckpd, FldName}
#    c_ij ---> R_i&{RecName} ---> RO_ij: R_ij&{RecName, Ckpd, FldName}
#    c_ij ---> RO_ij 


idx: 0
case: {'PID': 1024150, 'ObsDT': Timestamp('2023-11-28 15:55:59.016000'), 'PInvID': '1024150-007', 'RxID': '1024150-007-001'}
ROdf: {'EgmEdu-Af1W': None}

idx: 1
case: {'PID': 1013126, 'ObsDT': Timestamp('2023-06-15 02:29:33.640000'), 'PInvID': '1013126-009', 'RxID': '1013126-009-000'}
ROdf: {'EgmEdu-Af1W': Dataset({
    features: ['PID', 'PInvID', 'RxID', 'EgmEduID', 'DT'],
    num_rows: 1
})}

idx: 2
case: {'PID': 1034286, 'ObsDT': Timestamp('2023-09-29 13:40:46.160000'), 'PInvID': '1034286-002', 'RxID': '1034286-002-000'}
ROdf: {'EgmEdu-Af1W': None}

idx: 3
case: {'PID': 1012301, 'ObsDT': Timestamp('2023-06-07 13:23:15.061000'), 'PInvID': '1012301-008', 'RxID': '1012301-008-001'}
ROdf: {'EgmEdu-Af1W': None}

idx: 4
case: {'PID': 1038570, 'ObsDT': Timestamp('2023-06-13 15:59:16.918000'), 'PInvID': '1038570-008', 'RxID': '1038570-008-004'}
ROdf: {'EgmEdu-Af1W': None}

idx: 5
case: {'PID': 1032487, 'ObsDT': Timestamp('2023-09-01 08:39:11.971000'), 'PInvID': '1032487-000', 'RxID'

In [45]:
# RO_to_ROInfo: RecObsName_to_RecObsInfo
[i for i in RecObsName_to_RecObsInfo]

['EgmEdu-Af1W']

In [46]:
CaseTknVocab

{'tkn': {'tid2tkn': {0: 'unk',
   1: 'InvBtnEgm',
   2: 'InvBtnEgmClicks',
   3: 'InvMinsUntil1stEgm',
   4: 'RxBtnEgm',
   5: 'RxBtnEgmClicks',
   6: 'RxMinsUntil1stEgm'},
  'tkn2tid': {'unk': 0,
   'InvBtnEgm': 1,
   'InvBtnEgmClicks': 2,
   'InvMinsUntil1stEgm': 3,
   'RxBtnEgm': 4,
   'RxBtnEgmClicks': 5,
   'RxMinsUntil1stEgm': 6}}}

In [47]:
idx = 7

case_example = idx_to_examples[idx]
print('case_example:', case_example)
RecObsName_to_RecObsDS = idx_to_RO_to_ROdf[idx] # RO_to_ROdf # RO_ij
print('RecObsName_to_RecObsDS (RO_ij):', RecObsName_to_RecObsDS)
print([i for i in RecObsName_to_RecObsInfo])
print(CaseTknVocab)

case_example: {'PID': 1042660, 'ObsDT': Timestamp('2023-09-27 17:42:56.103000'), 'PInvID': '1042660-012', 'RxID': '1042660-012-000'}
RecObsName_to_RecObsDS (RO_ij): {'EgmEdu-Af1W': None}
['EgmEdu-Af1W']
{'tkn': {'tid2tkn': {0: 'unk', 1: 'InvBtnEgm', 2: 'InvBtnEgmClicks', 3: 'InvMinsUntil1stEgm', 4: 'RxBtnEgm', 5: 'RxBtnEgmClicks', 6: 'RxMinsUntil1stEgm'}, 'tkn2tid': {'unk': 0, 'InvBtnEgm': 1, 'InvBtnEgmClicks': 2, 'InvMinsUntil1stEgm': 3, 'RxBtnEgm': 4, 'RxBtnEgmClicks': 5, 'RxMinsUntil1stEgm': 6}}}


# [Step 10]: Develop $\Phi$. 

In [48]:
def fn_CaseTkn(case_example, 
               RecObsName_to_RecObsDS, 
               RecObsName_to_RecObsInfo, 
               CaseTknVocab):
    
    # input: RecObsName_to_RecObsDS, RecObsName_to_RecObsInfo
    # output: CaseObservation
    assert len(RecObsName_to_RecObsDS) == 1
    RecObsName = [i for i in RecObsName_to_RecObsDS][0]
    RecObsDS   = RecObsName_to_RecObsDS[RecObsName]

    if RecObsDS is None or len(RecObsDS) == 0:
        result_dict = {'InvBtnEgm': 0, 
                       'RxBtnEgm': 0}
    else: 
        RxIDValue = case_example['RxID']
        ObsDTValue = case_example['ObsDT']
        df_InvEgm = RecObsDS.to_pandas()
        time_until_firstclick = (df_InvEgm['DT'].min() - ObsDTValue).total_seconds() / 60
        result_dict = {'InvBtnEgm': 1,  
                       'InvBtnEgmClicks': len(df_InvEgm), 
                       'InvMinsUntil1stEgm': time_until_firstclick}
        # print(df_InvEgm)
        # print(case_example['RxID'])
        df_RxEgm = df_InvEgm[df_InvEgm['RxID'] == RxIDValue].reset_index()
        if len(df_RxEgm) == 0:
            result_dict['RxBtnEgm'] = 0
        else:
            time_until_firstclick = (df_RxEgm['DT'].min() - ObsDTValue).total_seconds() / 60
            result_dict['RxBtnEgm'] = 1
            result_dict['RxBtnEgmClicks'] = len(df_RxEgm)
            result_dict['RxMinsUntil1stEgm'] = time_until_firstclick
            
    result_dict = {'tkn': [i for i in result_dict], 'wgt': [float(round(result_dict[i], 2)) for i in result_dict]}
    result_dict['tid'] = [CaseTknVocab['tkn']['tkn2tid'][tkn] for tkn in result_dict['tkn']]
    CaseObservation = {'tid': result_dict['tid'], 'wgt': result_dict['wgt']}
    return CaseObservation

fn_CaseTkn.fn_string = inspect.getsource(fn_CaseTkn)

In [49]:
fn_CaseTkn(case_example, # (pi, tij)
            RecObsName_to_RecObsDS, # RO_ij^{name, ckpd, \phi} 
            RecObsName_to_RecObsInfo, 
            CaseTknVocab)


{'tid': [1, 4], 'wgt': [0.0, 0.0]}

## Save Files

In [50]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [get_caseobs_id, get_selected_columns, get_casetkn_vocab, fn_CaseTkn]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casetkn', f'casetkn_{CaseTkn}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [51]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer

# get_caseobs_id, get_selected_columns, get_casetkn_vocab, fn_CaseTkn
batch_size = 100

CaseObsName = convert_RecObsName_and_CaseTkn_to_CaseObsName(Record_Observations_List, CaseTkn)
RecObsName_to_RecObsInfo = get_RecObsName_to_RecObsInfo(Record_Observations_List, 
                                                        CaseTkn, get_selected_columns, 
                                                        cohort_args, Ckpd_ObservationS)
fn_caseobs_Phi = CaseObserverTransformer(RecObsName_to_RecObsInfo, 
                                         CaseTkn, 
                                         get_casetkn_vocab, 
                                         fn_CaseTkn, 
                                         get_caseobs_id, 
                                         use_caseobs_from_disk = True, 
                                         CaseObsFolder = CaseObsFolder,
                                         caseobs_ids = None
                                         )

CaseTknVocab = fn_caseobs_Phi.CaseTknVocab # get_casetkn_vocab(RecObsName_to_RecObsInfo)
fn_caseobs_Phi.new_calculated_caseobs

{}

In [52]:
Ckpd_ObservationS

{'InObs': {'CkpdName': 'InCase',
  'DistStartToPredDT': -0.01,
  'DistEndToPredDT': 0.01,
  'TimeUnit': 'min',
  'StartIdx5Min': 0,
  'EndIdx5Min': 0},
 'Bf24H': {'CkpdName': 'Bf24H',
  'DistStartToPredDT': -24,
  'DistEndToPredDT': 0.01,
  'TimeUnit': 'H',
  'StartIdx5Min': -288,
  'EndIdx5Min': 0},
 'Bf1M': {'CkpdName': 'Bf1M',
  'DistStartToPredDT': -720,
  'DistEndToPredDT': 0.01,
  'TimeUnit': 'H',
  'StartIdx5Min': -8640,
  'EndIdx5Min': 0},
 'Af2H': {'CkpdName': 'Af2H',
  'DistStartToPredDT': 1,
  'DistEndToPredDT': 121,
  'TimeUnit': 'min',
  'StartIdx5Min': 1,
  'EndIdx5Min': 24},
 'Af1W': {'CkpdName': 'Af1W',
  'DistStartToPredDT': 0.0001,
  'DistEndToPredDT': 7,
  'TimeUnit': 'D',
  'StartIdx5Min': 1,
  'EndIdx5Min': 2016}}

In [53]:
CaseObsName

'ro.EgmEdu-Af1W_ct.FutRxEduTkn'

In [54]:
RecObsName_to_RecObsInfo

{'EgmEdu-Af1W': {'rec_args': {'CohortInfo': {'RawData2023Nov_Trulicity': {'EgmBase': {'raw_data_path': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity/Egm_CampaignEducationalContentTap_df_de_identified.csv',
      'RawRootID': 'patient_id_encoded',
      'RecNumColumn': 'Egm_CampaignEducationalContentTap',
      'raw_columns': ['patient_id_encoded',
       'invitation_id_encoded',
       'prescription_id_encoded',
       'event_id',
       'invitation_date',
       'event_date',
       'p_created_date',
       'created_date',
       'external_id',
       'campaign_id',
       'event_name',
       'key1',
       'value1',
       'key2',
       'value2',
       'key3',
       'value3',
       'key4',
       'value4',
       'key5',
       'value5',
       'server_id',
       'partner_id',
       'content_id',
       'basket_id',
       'workflow_step',
       'user_agent',
       'record_locator_encoded']}}},
   'RecName': 'EgmEdu',
   'RecID': 'EgmEduID',
   'RecIDChain': ['PID', 'PInv

In [55]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                            batched = True, 
                            batch_size= batch_size, 
                            load_from_cache_file=False, 
                            new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/40517 [00:00<?, ? examples/s]

Map: 100%|██████████| 40517/40517 [00:02<00:00, 13825.84 examples/s]

Elipse Time:  0:00:02.940074
Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'tid', 'wgt'],
    num_rows: 40517
})


In [56]:
print(len(fn_caseobs_Phi.new_calculated_caseobs))

40517


In [57]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                            batched = True, 
                            batch_size= batch_size, 
                            load_from_cache_file=False, 
                            new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map: 100%|██████████| 40517/40517 [00:00<00:00, 87827.83 examples/s]

Elipse Time:  0:00:00.464475
Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'tid', 'wgt'],
    num_rows: 40517
})


In [58]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int])
print(CaseTknVocab)

ro.EgmEdu-Af1W_ct.FutRxEduTkn
12080
{'ObsDT': Timestamp('2023-09-25 15:57:55.154000'),
 'PID': 1013776,
 'PInvID': '1013776-013',
 'RxID': '1013776-013-004',
 'tid': [1, 4],
 'wgt': [0.0, 0.0]}
{'tkn': {'tid2tkn': {0: 'unk', 1: 'InvBtnEgm', 2: 'InvBtnEgmClicks', 3: 'InvMinsUntil1stEgm', 4: 'RxBtnEgm', 5: 'RxBtnEgmClicks', 6: 'RxMinsUntil1stEgm'}, 'tkn2tid': {'unk': 0, 'InvBtnEgm': 1, 'InvBtnEgmClicks': 2, 'InvMinsUntil1stEgm': 3, 'RxBtnEgm': 4, 'RxBtnEgmClicks': 5, 'RxMinsUntil1stEgm': 6}}}


In [59]:
ds_casetkn[762]

{'PID': 1000924,
 'ObsDT': Timestamp('2023-03-15 19:13:21.749000'),
 'PInvID': '1000924-001',
 'RxID': '1000924-001-000',
 'tid': [1, 2, 3, 4],
 'wgt': [1.0, 1.0, 1785.33, 0.0]}

## Save

In [60]:
fn_caseobs_Phi.df_caseobs_info

,caseobs_idx_in_data
caseobs_id,


In [61]:
fn_caseobs_Phi.save_new_caseobs_to_ds_caseobs()

Saving the dataset (1/1 shards): 100%|██████████| 40517/40517 [00:00<00:00, 3998132.34 examples/s]


In [62]:
fn_caseobs_Phi.df_caseobs_info

,caseobs_idx_in_data
caseobs_id,


In [63]:
fn_caseobs_Phi.ds_caseobs_data

# Save Vocab

In [64]:
CaseTknVocab = fn_caseobs_Phi.CaseTknVocab
CaseTknVocab

{'tkn': {'tid2tkn': {0: 'unk',
   1: 'InvBtnEgm',
   2: 'InvBtnEgmClicks',
   3: 'InvMinsUntil1stEgm',
   4: 'RxBtnEgm',
   5: 'RxBtnEgmClicks',
   6: 'RxMinsUntil1stEgm'},
  'tkn2tid': {'unk': 0,
   'InvBtnEgm': 1,
   'InvBtnEgmClicks': 2,
   'InvMinsUntil1stEgm': 3,
   'RxBtnEgm': 4,
   'RxBtnEgmClicks': 5,
   'RxMinsUntil1stEgm': 6}}}

In [89]:
pd.DataFrame({CaseObsName: CaseTknVocab}).to_pickle(fn_caseobs_Phi.CaseObsFolder_vocab)